In [142]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

In [27]:
node_columns = [
    "tax_id",
    "parent tax_id",
    "rank",
    "embl code",
    "division id",
    "inherited div flag",
    "genetic code id",
    "inherited GC  flag",
    "mitochondrial genetic code id",
    "inherited MGC flag",
    "GenBank hidden flag",
    "hidden subtree root flag",
    "comments"
]

In [31]:
name_columns = [
    "tax_id",
	"name_txt",
	"unique name",
	"name class"  
    ]

In [158]:
df_name = pd.read_csv(
    "/Users/siomi19/Downloads/taxdump/names.dmp",
    sep="\t",
    header=None,
    index_col=0,
    usecols=np.arange(0,7,2),
    names=name_columns
)
df_name = df_name[df_name["name class"]=="scientific name"]

df_node = pd.read_csv(
    "/Users/siomi19/Downloads/taxdump/nodes.dmp",
    sep="\t",
    header=None,
    index_col=0,
    usecols=np.arange(0,25,2),
    names=node_columns
).loc[df_name.index]

/Users/siomi19/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/Users/siomi19/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [164]:
def get_lineage(row):
    parents = []
    def get_parent(tax_id):
        try:
            parents.append(df_name.loc[df_node.loc[tax_id, "parent tax_id"], "name_txt"])
            if df_node.loc[tax_id, "parent tax_id"] == tax_id:
                return parents
            return get_parent(df_node.loc[tax_id, "parent tax_id"])
        except KeyError:
            return parents
    get_parent(row.name)
    row["lineage"] = ",".join(parents[:-1])
    return row

In [165]:
df_node_ =  df_node.progress_apply(lambda x:get_lineage(x), axis=1)
df_node_.to_csv(f"/Users/siomi19/Desktop/comprehensive_PrLD_prediction/nodes_lineage.txt", sep="\t")

100%|██████████| 2468672/2468672 [1:20:54<00:00, 508.57it/s] 
